# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World


## Instructions

Total Points: Undergrads 100 / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a PDF file. 

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [2]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [3]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'north'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`. 

In [4]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [5]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: east
step 1 - action: east
step 2 - action: east
step 3 - action: west
step 4 - action: north
step 5 - action: west
step 6 - action: south
step 7 - action: west
step 8 - action: west
step 9 - action: west
step 10 - action: west
step 11 - action: west
step 12 - action: suck
step 13 - action: east
step 14 - action: west
step 15 - action: north
step 16 - action: west
step 17 - action: south
step 18 - action: west
step 19 - action: north


1

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement inteligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formated professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. Use the same environmnt for all your agent implementations in the tasks below.

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

This environment is configured for when the wall obstacles are added. It has two variables to keep track of the dirty spots in the area (which is randomly generated on line 7). If walls are enabled, then they are also generated on line 7 as well. The "`room`" (or space) is configured per tile as follows:

   0. Values of 0 in the array correspond to empty tiles
   1. Values of 1 in the array signify a dirty floor tile
   2. Values of 2 are wall obstacles and cannot be moved into or through

The `make_move()` function serves as a bounds checking method to make sure that the agent can move into a certain tile, or updates the dirty tile count if that is the action taken. If `verbose` is `True`, then it also serves as a record of what the agent has done.


In [16]:
import random
# Your code and description goes here
def simulation_environment(agent, max_steps, size = 5, walls = False, verbose = True):

    global room
    room = np.random.randint(3 if walls else 2,size=(size,size))        #creating the space
    global initial_dirty
    initial_dirty = np.count_nonzero(room==1)                           #setting the maximum
                                                                        #amount of dirty spaces
    global current_dirty
    current_dirty = initial_dirty                                       #used to track
                                                                        #current amount of dirty
                                                                        #squares
    global position
    position = (random.randrange(0,size),random.randrange(0,size))      #stores the position of
                                                                        #the agent
    while(room[position]==2):
        position = (random.randrange(0,size),random.randrange(0,size))  #makes sure the agent
                                                                        #does not initialize on
                                                                        #top of a wall

    def make_move(action):                                              # This next loop helps
        global current_dirty,position,room                              # determine if the next
        if action == "suck":                                            # made move is legal or
            if(verbose):print("suck",end=' ')                           # if it would put the
            if room[position]==1:                                       # agent in a wall or out
                current_dirty = current_dirty-1                         # of bounds
                room[position] = 0
                if(verbose):print("success")
            else:
                if(verbose):print("failed")
        if action == "north":
            if(verbose):print("moving north",end=' ')
            if position[0]!=0 and room[position[0]-1,position[1]]!=2:
                position = (position[0]-1,position[1])
                if(verbose):print("",)
            else:
                if(verbose):print("blocked")
        if action == "south":
            if(verbose):print("moving south",end=' ')
            if position[0]!=size-1 and room[position[0]+1,position[1]]!=2:
                position = (position[0]+1,position[1])
                if(verbose):print("",)
            else:
                if(verbose):print("blocked")
        if action == "east":
            if(verbose):print("moving east",end=' ')
            if position[1]!=size-1 and room[position[0],position[1]+1]!=2:
                position = (position[0],position[1]+1)
                if(verbose):print("",)
            else:
                if(verbose):print("blocked")
        if action == "west":
            if(verbose):print("moving west",end=' ')
            if position[1]!=0 and room[position[0],position[1]-1]!=2:
                position = (position[0],position[1]-1)
                if(verbose):print("",)
            else:
                if(verbose):print("blocked")


    for i in range(max_steps):
        dirty = room[position]
        bumpers = {"north" : True if position[0]==0 else room[position[0]-1,position[1]]==2,
                   "south" : True if position[0]==size-1 else room[position[0]+1,position[1]]==2,
                   "west" : True if position[1]==0 else room[position[0],position[1]-1]==2,
                   "east" : True if position[1]==size-1 else room[position[0],position[1]+1]==2}



        if verbose:print("Step",i,"\nat",position)          #some printing code
        action = agent(bumpers,dirty)
        make_move(action)
    if(verbose):print("\n\n\n")
    if(verbose):print("Cleaned: ",initial_dirty-current_dirty,"/",initial_dirty,"\nSuccess rate:",float(100*(initial_dirty-current_dirty)/initial_dirty),"%",sep="")
    if(verbose):print("returning",(initial_dirty - current_dirty) / initial_dirty)
    return (initial_dirty - current_dirty) / initial_dirty

## Task 2:  Implement a simple reflex agent [10 Points] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

This code is very simple. The engine of the agent is the `move()` function that randomly decides where to go

In [7]:
class simple_reflex():
    def __init__(self,verbose=True):
        self.verbose = verbose

    def move(self, bumpers, dirty):
        if dirty:
            if(self.verbose):print("suck")      #this part of the loop just immediately looks for
            return "suck"                       #a dirty tile and cleans it before anything else


        else:
            temp = self.choice(bumpers)
            while((temp=="north" and bumpers['north']==True) or     #this loop randomly selects a
                  (temp=="south" and bumpers['south']==True) or     #choice, but if the bumper for
                  (temp=="east" and bumpers['east']==True) or       #that direction is on, then it
                  (temp=="west" and bumpers['west']==True)):        #re-selects a direction
                temp = self.choice(bumpers)

            if(self.verbose):print(temp)
            return temp                     #returns the selection


    def choice(self,bumpers):
        return np.random.choice(["north", "east", "west", "south"]) #randomly chooses a direction


# verbose = False
# reflex_agent = simple_reflex(verbose=verbose)
# print(simulation_environment(reflex_agent.move,100,verbose=verbose)) if verbose else simulation_environment(reflex_agent.move,100,verbose=verbose)

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb))

The Agent state is just an int, it begins in the 0 position, signifying that it has not been fully calibrated yet. it will progress up until it reaches phase 3, and then use the rest of its energy sucking if there is any left over. The phase values and their codes are as follows:

0. Move north until it finds the north wall
1. Move west until it finds the northwest corner
2. Move alternating north/south by column until it finds the wall, then move one to the east and continue
3. It has now found the east wall and prepares to make its final column traversal

In [8]:
class model_reflex():
    def __init__(self,steplimit=100,verbose = True):
        self.verbose = verbose
        self.phase = 0
        self.direction = "north"
        self.next_direction = ""
        self.steplimit = steplimit

    def move(self,bumpers,dirty):                                   # this helps move the agent,
        if dirty:                                                   # the first priority is
            return "suck"                                           # cleaning, next is moving
        if self.phase == 0 and bumpers['north']:                    # to the northwest corner
            self.direction = "west"                                 # and then moving up and down
            self.phase = 1                                          # by rows
            if(self.verbose):print("setting phase to 0")
        if self.phase == 1 and bumpers['north'] and bumpers['west']:
            self.direction = "south"
            self.phase = 2
            if(self.verbose):print("setting phase to 1")
        elif self.phase == 2:
            if bumpers['east']:
                self.direction=self.next_direction
                self.next_direction=""
                self.phase=3
            elif self.next_direction!="":
                self.direction=self.next_direction
                self.next_direction=""
            elif bumpers['north']:
                self.direction="east"
                self.next_direction="south"
            elif bumpers['south']:
                self.direction="east"
                self.next_direction="north"
        elif self.phase == 3:
            if self.next_direction!="":
                self.direction=self.next_direction
                self.next_direction=""
            elif bumpers['east'] and (bumpers['north'] or bumpers['south']):
                self.next_direction = ""
                return "suck"


        return(self.direction)
agent = model_reflex(verbose=False)
print(simulation_environment(agent.move,100,verbose=False))

1.0


## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents. 
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb))

In [14]:
rand_agent_avg = 0.0
simple_agent_avg = 0.0
model_agent_avg = 0.0

steps = 10000
size = 100

for i in range(100):
    reflex_agent = simple_reflex(verbose=False)
    model_agent = model_reflex(verbose=False)

    rand_agent_avg = ((i*rand_agent_avg)+simulation_environment(simple_randomized_agent,max_steps=steps,verbose=False,size=size))/(i+1) if i !=0 else simulation_environment(simple_randomized_agent,max_steps=steps,verbose=False,size=size)
    simple_agent_avg = ((i*simple_agent_avg)+simulation_environment(reflex_agent.move,max_steps=steps,verbose=False,size=size))/(i+1) if i !=0 else simulation_environment(reflex_agent.move,max_steps=steps,verbose=False,size=size)
    model_agent_avg = ((i*model_agent_avg)+simulation_environment(model_agent.move,max_steps=steps,verbose=False,size=size))/(i+1) if i !=0 else simulation_environment(model_agent.move,max_steps=steps,verbose=False,size=size)


print(int(rand_agent_avg*100),"% ",int(simple_agent_avg*100),"% ",int(model_agent_avg*100),"% ",sep="",end="\n")

9% 21% 66% 


Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size      | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent | Steps  |
|-----------|------------------|---------------------|--------------------------|--------|
| 5x5       | 14%              | 43%                 | 61%                      | 25     |
| 10x10     | 11%              | 32%                 | 63%                      | 100    |
| 100x100   | 9%               | 21%                 | 66%                      | 10,000 |

Add charts to compare the performance of the different agents.

My graphs seemed to not be able to show up, so I will just talk about them instead.

I made a bar graph showing the three agents versus the three sizes, and noted that the randomized agent stayed at around 17% of the performance of the model based reflex agent, and that as the size increased, the percent of dirt cleared decreased for the randomized and simple reflex   agent decreased.
Now, I defined the step count of each map size based off of getting the model based reflex agent being around 65% with a clean step count, so the increasing percentage of the model-based agent is purely coincidental. Nevertheless, the more space there is, the less was cleaned by the random and simple agents.

## Task 5: Robustness of the agent implementations [10 Points] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

# Random Agent
Generally speaking, for the random agent, there is no difference if there is a wall in any direction or not. This agent decides its move indiscriminately of the options proposed so the layout of the map or room does not have any affect on its choice making abilities.

# Simple Reflex Agent
For the simple reflex agent, it will still look around randomly, but it will not waste time running into walls, so it should perform relatively well. This is similar to the random agent, but it does not spend time bumping into walls and still moves around pseudorandomly.

# Model Agent
For the model agent, it will actually have its search area severely limited, so it will likely run less effectively than the other agents, but that is why I implemented the second model reflex agent below.

## Graduate student advanced task: Obstacles [10 Points]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus Points].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

## New Model-Based Reflex Agent
It is now based on least "cost" of an action (not the one energy cost of moving or cleaning, but a term that describes the most informed choice) and finds its next move based on what makes the most sense. There are a few criterion that I implemented that change the appeal of each move:

### Positive
1. Maintaining the same direction of movement
2. Moving away from the corner

### Negative
1. A wall is in the way
2. The agent has already traveled to that tile


## Other ideas
### Positive
1. checking corners or hallways of a map that have not been explored yet
2. Pathfinding through a room with a relatively known layout

### Negative
1. subdividing the known internal map into areas occasionally and seeing if the next move would take it into a space where it would have to double back

These together will help the agent decide its next move, the next step after that would be implementing code that would help decide two moves in advance, but that code is much more complicated than the scope of this project.

In [21]:
class model_reflex():
    def __init__(self, steplimit=100, verbose=True):
        self.verbose = verbose
        self.direction = "north"
        self.next_direction = ""
        self.map = np.zeros((steplimit, steplimit))
        self.x = 0
        self.y = 0
        self.steplimit = steplimit

    def move(self,bumpers,dirty):
        if dirty:
            if(self.verbose):print("suck")      #this part of the loop just immediately looks for
            return "suck"                       #a dirty tile and cleans it before anything else

        directions = {'north':100, 'west': 100, 'south': 100, 'east': 100}
        self.record(bumpers)                    #populates the internal map with all walls

        directions = {k:directions[k]-(5 if k == self.direction else 0) for k in directions}
        directions = {k:directions[k]+(20 if bumpers[k] else 0) for k in directions}
        directions = {k:directions[k]+(10 if self.map[self.x,self.y]==1 else 0) for k in directions}


        if(bumpers['north']):                   # choosing the next move based on if it is in the
            if(bumpers['east']):                # corner
                directions['west']-=6
                directions['south']-=6
            if(bumpers['west']):
                directions['east']-=6
                directions['south']-=6
        if(bumpers['south']):
            if(bumpers['east']):
                directions['west']-=6
                directions['north']-=6
            if(bumpers['west']):
                directions['east']-=6
                directions['north']-=6
        if(bumpers['east']):
            if(bumpers['north']):
                directions['west']-=6
                directions['south']-=6
            if(bumpers['south']):
                directions['west']-=6
                directions['north']-=6
        if(bumpers['west']):
            if(bumpers['south']):
                directions['west']-=6
                directions['north']-=6
            if(bumpers['north']):
                directions['south']-=6


        if(self.verbose):print(bumpers)
        if(self.verbose):print(directions)
        if(self.verbose):print(list(dict(sorted(directions.items(), key=lambda x:x[1])).keys())[0])
        self.direction = (list(dict(sorted(directions.items(), key=lambda x:x[1])).keys())[0])
        self.map[self.x,self.y] = 1
        if self.direction=='north':
            self.x+=1
        if self.direction=='south':
            self.x-=1
        if self.direction=='east':
            self.y+=1
        if self.direction=='west':
            self.y-=1
        return (list(dict(sorted(directions.items(), key=lambda x:x[1])).keys())[0])

    def record(self,bumpers):
        if(bumpers['north']):
            self.map[self.x-1,self.y] = 2
        if(bumpers['south']):
            self.map[self.x+1,self.y] = 2
        if(bumpers['east']):
            self.map[self.x,self.y+1] = 2
        if(bumpers['west']):
            self.map[self.x,self.y-1] = 2

## More advanced implementation tasks

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actualy depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currebntly clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [13]:
# Your ideas/code